# Load data

In [33]:
import pandas as pd, duckdb, joblib, os, numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio # settings like template by default

from dotenv import load_dotenv
from pathlib import Path

load_dotenv(dotenv_path = Path('.') / '.config')
pio.templates.default = os.getenv("template_plotly") # default template
pio.renderers.default = 'notebook_connected'

from helper_functs.tsfuncts import create_features_extracted_days, calculate_metrics, df_average_error, split_sequence

In [22]:
# Define the path to your CSV file
csv_path = 'data/node_demand.csv'

# Use DuckDB to query only the needed columns
query = f"""
SELECT date_trunc('hour', time) AS time_value, MAX(nodes) AS max_nodes
FROM read_csv_auto('{csv_path}')
WHERE time >= '2023-10-23'
GROUP BY time_value
ORDER BY time_value
"""

# Execute the query and load the result into a Pandas DataFrame
df = duckdb.query(query).to_df()

# Display the DataFrame
print(df.shape)
print(df.head(5))

(9348, 2)
           time_value  max_nodes
0 2023-10-23 05:00:00          2
1 2023-10-23 06:00:00          3
2 2023-10-23 07:00:00          5
3 2023-10-23 08:00:00         11
4 2023-10-23 09:00:00         16


In [23]:
print(df.dtypes)
# data['time'] = data['time'].dt.floor('h')
# df = data.groupby('time')['nodes'].max().reset_index()
print(f"min date {min(df.time_value)} and max date {max(df.time_value)}")
df.set_index('time_value', inplace=True)
print(df.head(3))

time_value    datetime64[us]
max_nodes              int64
dtype: object
min date 2023-10-23 05:00:00 and max date 2025-06-03 11:00:00
                     max_nodes
time_value                    
2023-10-23 05:00:00          2
2023-10-23 06:00:00          3
2023-10-23 07:00:00          5


In [24]:
# Create a complete hourly time range
full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='h')
# Reindex to include all hours, filling missing with NaN
df = df.reindex(full_range)
print(df.isna().sum())
print(df[df['max_nodes'].isna()].head(5))

max_nodes    4795
dtype: int64
                     max_nodes
2023-10-23 23:00:00        NaN
2023-10-24 01:00:00        NaN
2023-10-24 04:00:00        NaN
2023-10-24 05:00:00        NaN
2023-10-25 01:00:00        NaN


In [25]:
# Complete NaN values
# df.fillna(0, inplace=True)
# Forward fill missing values
df = df.ffill()
print(df.isna().sum())
print(df[df.index >= pd.to_datetime("2023-10-23 22:00:00")].head(10))

max_nodes    0
dtype: int64
                     max_nodes
2023-10-23 22:00:00        2.0
2023-10-23 23:00:00        2.0
2023-10-24 00:00:00        2.0
2023-10-24 01:00:00        2.0
2023-10-24 02:00:00        3.0
2023-10-24 03:00:00        3.0
2023-10-24 04:00:00        3.0
2023-10-24 05:00:00        3.0
2023-10-24 06:00:00        3.0
2023-10-24 07:00:00        6.0


In [26]:
date_split = "2025-04-03"
df['train_test'] = (df.index >= pd.to_datetime("2025-04-03"))
fig = px.line(df, x = df.index, y = 'max_nodes', color = 'train_test')
fig.update_layout(
    title = f"<b>Time series from {min(df.index)} to {max(df.index)}</b>", 
    xaxis = dict(title='Date', tickangle=-45), yaxis = dict(title='nodes')
)
fig.show()

# Feature creation

In [27]:
df = create_features_extracted_days(df)
print(f"New dataframe has {df.shape}")
print(df.head(3))

New dataframe has (14143, 10)
                     max_nodes  train_test  hour  dayofweek  quarter  month  \
2023-10-23 05:00:00        2.0       False     5          0        4     10   
2023-10-23 06:00:00        3.0       False     6          0        4     10   
2023-10-23 07:00:00        5.0       False     7          0        4     10   

                     year  dayofyear  dayofmonth  weekofyear  
2023-10-23 05:00:00  2023        296          23          43  
2023-10-23 06:00:00  2023        296          23          43  
2023-10-23 07:00:00  2023        296          23          43  


In [8]:
fig = px.box(df, x='hour', y='max_nodes', color = 'hour')
fig.update_layout(
    title = "<b>Max nodes per hour</b>",
    xaxis = dict(title='Hour'), yaxis = dict(title='Energy')
)
fig.show()

In [9]:
fig = px.box(df, x='month', y='max_nodes', color = 'month')
fig.update_layout(
    title = "<b>Max nodes per month</b>",
    xaxis = dict(title='month'), yaxis = dict(title='max nodes'))
fig.show()

# Model

In [10]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, RandomizedSearchCV

train, test = df.loc[df.train_test == False], df.loc[df.train_test == True]

FEATURES = ['dayofyear', 'hour', 'dayofweek', 'month', 'quarter', 'year'] # 
TARGET = 'max_nodes'

# Train a test split
X_train, y_train = train[FEATURES], train[TARGET]
X_test, y_test = test[FEATURES], test[TARGET]

# Variables used in deep learning models
# my_batch, my_epochs, my_val = 24, 159, 0.12

# Define time series cross-validator
tscv = TimeSeriesSplit(n_splits=10)
# early_stop helps to stop training when using GridSearchCV for deep learning models
# early_stop = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss',     # Metric to monitor (e.g., 'val_loss' or 'val_accuracy')
#     patience=12,             # Number of epochs to wait for improvement
#     restore_best_weights=True  # Restore model weights from the epoch with the best value
# )
# neural network input and output
# n_input, n_output = X_train.shape[1], 1

print(f"X_train size {X_train.shape}, y_train size {y_train.shape}")
print(f"X_test size {X_test.shape}, y_test size {y_test.shape}")

X_train size (12667, 6), y_train size (12667,)
X_test size (1476, 6), y_test size (1476,)


# XGBoost

In [11]:
%%time

# Hyperparameters for xgboost
param_grid = {
    'base_score': [0.895, 0.896], 
    'booster': ['gbtree'],
    'n_estimators': [45, 40],
    'early_stopping_rounds': [55, 57],
    'colsample_bytree': [0.9, 0.95],
    'objective': ['reg:squarederror'],
    'max_depth': [7, 6, 5],
    'learning_rate': [0.255, 0.25],
    'reg_lambda': [3, 2, 1],
}

reg = xgb.XGBRegressor()

grid_search = GridSearchCV(estimator=reg, param_grid=param_grid, cv=tscv,
                            scoring='neg_mean_squared_error', verbose=False, n_jobs=-1)

grid_search.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=False)
best_reg_model = grid_search.best_estimator_ # best model
print("Best hyperparameters:", grid_search.best_params_)

Best hyperparameters: {'base_score': 0.895, 'booster': 'gbtree', 'colsample_bytree': 0.9, 'early_stopping_rounds': 55, 'learning_rate': 0.255, 'max_depth': 6, 'n_estimators': 45, 'objective': 'reg:squarederror', 'reg_lambda': 2}
CPU times: user 4.01 s, sys: 606 ms, total: 4.61 s
Wall time: 1min 14s


In [12]:
# Feature importance
fi = pd.DataFrame(data=best_reg_model.feature_importances_,
             index=best_reg_model.feature_names_in_, columns=['importance'])
fi.sort_values('importance', inplace = True)

fig = px.bar(fi, y = fi.index, x = 'importance', orientation='h')
fig.show()

In [13]:
filename = f"{os.getenv("MODELS_TS")}xgb_nodes.pkl"
joblib.dump(best_reg_model, filename)
!du -sh {filename} | cut -f1

207K


In [14]:
# Load the model
filename = f"{os.getenv("MODELS_TS")}xgb_nodes.pkl"
reg_model = joblib.load(filename)

y_pred = reg_model.predict(X_test)
dfres = df[['max_nodes']].tail(X_test.shape[0]).copy()
# dfres = dfres.merge(test[['xgbpred']], how='left', left_index=True, right_index=True)
dfres['pred_xgb'] = y_pred.flatten()
score = calculate_metrics(dfres.max_nodes, dfres.pred_xgb)
print(dfres.head(2))
fig = px.line(dfres, x = dfres.index, y=['max_nodes', 'pred_xgb'],
            labels={'value': 'Nodes', 'variable': 'Source'})
fig.update_layout(
    title=f'Nodes Predicted vs Used with {score}',
    xaxis = dict(title = "Dates", tickangle=-45), yaxis = dict(title='Electricity (kWh)')
)
fig.show()

                     max_nodes  pred_xgb
2025-04-03 00:00:00        2.0  1.475542
2025-04-03 01:00:00        2.0  1.475542


# Deep learning

In [15]:
import tensorflow as tf

only_cpu = True
if only_cpu: 
    os.environ["CUDA_VISIBLE_DEVICES"] = str(os.getenv("CUDA_VISIBLE_DEVICES"))
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = str(os.getenv("TF_CPP_MIN_LOG_LEVEL"))  # 0 = all logs, 1 = filter INFO, 2 = filter WARNING, 3 = filter ERROR
else:
    gpus = tf.config.list_physical_devices('GPU')  # obtiene las gpus instaladas
    if len(gpus) > 0:   # If GPU available
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print(gpus)

2025-06-06 13:57:43.883292: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749218263.894113   37307 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749218263.897530   37307 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## LSTM with lags

In [84]:
n_steps, n_features = 6, 1
train_date = pd.to_datetime(date_split) + pd.Timedelta(hours=n_steps-1)
compared_date = pd.to_datetime(train_date) + pd.Timedelta(hours=n_features)
train_date = pd.to_datetime(date_split) + pd.Timedelta(hours=n_steps-1)
train, test = df.loc[:train_date][["max_nodes"]], df.loc[date_split:][["max_nodes"]] # test set starts before the end of the train set
print(f"Train set: {train.shape} and Test set: {test.shape}")
print(f"Train maz date: {max(train.index)} and test min date {min(test.index)}")

Train set: (12673, 1) and Test set: (1476, 1)
Train maz date: 2025-04-03 05:00:00 and test min date 2025-04-03 00:00:00


In [91]:
# Variables used in deep learning models
my_batch, my_epochs, my_val = 12, 200, 0.2
print(f"n epochs {my_epochs}")
# early_stop helps to stop training when using GridSearchCV for deep learning models
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',     # Metric to monitor (e.g., 'val_loss' or 'val_accuracy')
    patience=12,             # Number of epochs to wait for improvement
    restore_best_weights=True  # Restore model weights from the epoch with the best value
)

n epochs 200


In [86]:
%%time
# Generate train sample
X_train, y_train = split_sequence(train.values, n_steps, n_features)
print(f"train shape: {X_train.shape, y_train.shape}")
# Generate test sample
X_test, y_test = split_sequence(test.values, n_steps, n_features)
print(f"test shape: {X_test.shape, y_test.shape}")

# Show the first sample
for i in range(1):
    print(X_train[i].tolist(), y_train[i])
    print(X_test[i].tolist(), y_test[i])

train shape: ((12667, 6, 1), (12667, 1))
test shape: ((1470, 6, 1), (1470, 1))
[[2.0], [3.0], [5.0], [11.0], [16.0], [18.0]] [24.]
[[2.0], [2.0], [2.0], [2.0], [2.0], [2.0]] [3.]
CPU times: user 16.3 ms, sys: 0 ns, total: 16.3 ms
Wall time: 15.9 ms


In [92]:
%%time
# Define a CNN model
model1 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(n_steps, n_features)),
    tf.keras.layers.LSTM(60, return_sequences=True),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.LSTM(30),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(7, activation = 'relu'),
    tf.keras.layers.Dense(1)
])

model1.compile(optimizer='adam', loss='mse')
# Train data
model1.fit(X_train, y_train, epochs=my_epochs, verbose=False, 
            validation_split=my_val, batch_size=my_batch, callbacks=[early_stop])

best_epoch = np.argmin(model1.history.history['val_loss']) + 1
fig = px.line(pd.DataFrame(model1.history.history), y='loss', title=f'Loss til epoch {best_epoch}')
fig.show()

CPU times: user 3min 12s, sys: 25 s, total: 3min 37s
Wall time: 1min 46s


In [93]:
updated_array = X_test[0:1, :, :]
list_predicted = []
for index in range(X_test.shape[0]):
    yhat = model1.predict(updated_array, verbose=False)  # predice el punto siguiente
    list_predicted.append(yhat[0, 0].tolist())
    # print(f"Predicted value: {yhat.shape} {yhat.tolist()}")
    updated_array = np.concatenate((updated_array[:, 1:, :], yhat.reshape(1, 1, 1)), axis=1)
print(f"Predicted values: {len(list_predicted)}")
print(f"y_test shape {y_test.shape}")
# rmse = np.sqrt(np.mean((y_test - np.array(list_predicted))**2))
score = calculate_metrics(y_test, np.array(list_predicted))
print("Score:", score)

Predicted values: 1470
y_test shape (1470, 1)
Score: {'RMSE': np.float64(15.49), 'MAE': 11.82}


In [94]:
compared_df = test[compared_date:].copy()
compared_df['Predicted'] = list_predicted
print(f"Results shape {compared_df.shape}")
print(compared_df.head(2))
# rmse = np.sqrt(np.mean((compared_df.Passengers - compared_df.Predicted)**2))
score = calculate_metrics(compared_df.max_nodes, compared_df.Predicted)
fig = px.line(compared_df, x = compared_df.index, y= ['max_nodes', 'Predicted'],
            labels = {'value': 'Number of passengers', 'variable': 'Legend'},
            title = f'Prediction with a CNN model (one step) score: {score}')
fig.show()

Results shape (1470, 2)
                     max_nodes  Predicted
2025-04-03 06:00:00        3.0   2.050218
2025-04-03 07:00:00        6.0   2.125113
